In [89]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as Func

import matplotlib.pyplot as plt

from tqdm import tqdm

In [90]:
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [91]:
path='./Iris.csv'
fullData=pd.read_csv(path)
fullData=fullData.sample(frac=1)
fullData[0:4]

# dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True, num_workers = 4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
132,133,6.4,2.8,5.6,2.2,Iris-virginica
69,70,5.6,2.5,3.9,1.1,Iris-versicolor
146,147,6.3,2.5,5.0,1.9,Iris-virginica
56,57,6.3,3.3,4.7,1.6,Iris-versicolor


In [92]:
fullData=fullData.drop(columns='Id')
clases=fullData.iloc[:,4].unique()
print(clases)

['Iris-virginica' 'Iris-versicolor' 'Iris-setosa']


In [93]:
newLabel={
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
fullData.replace(newLabel,inplace=True)

In [94]:
fullData

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
132,6.4,2.8,5.6,2.2,2
69,5.6,2.5,3.9,1.1,1
146,6.3,2.5,5.0,1.9,2
56,6.3,3.3,4.7,1.6,1
1,4.9,3.0,1.4,0.2,0
...,...,...,...,...,...
40,5.0,3.5,1.3,0.3,0
64,5.6,2.9,3.6,1.3,1
4,5.0,3.6,1.4,0.2,0
39,5.1,3.4,1.5,0.2,0


In [95]:

CountOfRows=fullData.shape[0]
testData=fullData[0:CountOfRows//3]
trainData=fullData[CountOfRows//3:CountOfRows]



In [96]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
    return in_file

In [97]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,csv_file):
        data=csv_file
        x=data.iloc[:,:4]
        x=normalizeCollumns(x)
        y=data.iloc[:,4]
        self.x_train=torch.Tensor(np.array(x))
        self.y_train=nn.functional.one_hot(torch.Tensor(np.array(y)).to(torch.int64),num_classes=3)
        
        
    def __len__(self):
        return len(self.y_train)
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [107]:
trainDataset=MyDataset(trainData)
trainDataloader= torch.utils.data.DataLoader(trainDataset,batch_size=5)

(tensor([0.1429, 0.1364, 0.5932, 0.6667]), tensor([0, 0, 1]))

In [100]:
print(trainData.shape)
print(testData.shape)

(100, 5)
(50, 5)


In [101]:
trainDataloader=torch.utils.data.Dataloader()

AttributeError: module 'torch.utils.data' has no attribute 'Dataloader'

In [ ]:
class SimpleConvNet(nn.Module):
    def __init__(self, base_nc=32):
        super().__init__()

        # self.flatten =nn.Flatten()
        # self.Linear1=nn.Linear(784,100)
        # self.act=nn.Sigmoid()
        # self.LInear2=nn.Linear(100,10)
        maxpool=nn.MaxPool2d(2,2)
        avgpool=nn.AdaptiveAvgPool2d((1,1))
        act = nn.ReLU()
        Linear1 = nn.Linear(4*base_nc,2)

        flatten = nn.Flatten()

        cna1_1=CNA(3,base_nc)
        cna1_2=CNA(base_nc,base_nc)

        cna2_1=CNA(base_nc,2*base_nc)
        cna2_2=CNA(2*base_nc, 2*base_nc)

        cna3_1=CNA(2*base_nc,4*base_nc)
        cna3_2=CNA(4*base_nc,4*base_nc)
        
        cna4_1=CNA(4*base_nc,4*base_nc)
        cna4_2=CNA(4*base_nc,4*base_nc)
        
        cna5_1=CNA(4*base_nc,4*base_nc)
        cna5_2=CNA(4*base_nc,4*base_nc)

        
        self.model = nn.Sequential(
            cna1_1,cna1_2,maxpool,
            cna2_1,cna2_2,maxpool,
            cna3_1,cna3_2,maxpool,
            cna4_1,cna4_2,maxpool,
            cna5_1,cna5_2,avgpool,
            flatten,Linear1
        )

    def forward(self, x):
        
        return self.model(x)
    


In [ ]:
#loss_fn=nn.BCEWithLogitsLoss().to(device)
loss_fn = nn.CrossEntropyLoss()
#loss_fn=nn.MSELoss()
optimizer=torch.optim.Adam(net.classifier.parameters(),lr=1e-4, betas=(0.9, 0.999))

In [ ]:
def get_accuracy(pred, label):
    answer = pred.detach().numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()


In [ ]:
epochs=10
for epoch in range(epochs):
    loss_val=0
    acc_val=0

    for sample in tqdm(dataloader):
        img,label = sample

        label = label.to(device)
        label = Func.one_hot(label, num_classes=2).float()
        # label=Func.one_hot(label,num_classes=10)
        #label = label.float()
        #label = label.unsqueeze(1)
        img = img.to(device)
        
        optimizer.zero_grad()
        pred = net (img)

        loss = loss_fn(pred, label)
        loss.backward()

        optimizer.step()
        
        loss_val+=loss.item()
        acc_val += get_accuracy(pred.cpu(), label.cpu())
    loss_val /= len(dataloader)
    acc_val /= len(dataloader)
    print("loss=",loss_val ,"accruracy=",acc_val)
    

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:26<00:00, 18.67it/s]


loss= 0.3366883409222562 accruracy= 0.8487774451097805


100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:26<00:00, 19.01it/s]


loss= 0.25963227789172394 accruracy= 0.8878493013972056


100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:26<00:00, 18.99it/s]


...

loss= 0.048343443951918974 accruracy= 0.9869011976047904


100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:26<00:00, 18.99it/s]


loss= 0.024691427787288348 accruracy= 0.9950099800399201


100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:26<00:00, 18.96it/s]

loss= 0.025200452388572648 accruracy= 0.9935129740518962


In [ ]:
testloader=torch.utils.data.DataLoader(testset, batch_size=1,shuffle=True, num_workers = 4)

In [ ]:
acc_val=0
for sample in testloader:
    img,label = sample
    #plt.imshow(img.numpy().transpose((1,2,0)))
    img = img.to(device)
    label = label.to(device)
    label = Func.one_hot(label, num_classes=2).float()
    label = label.unsqueeze(1)
    pred = net(img)
    acc_val += get_accuracy(pred.cpu(), label.cpu())
    #print(acc_val)
    #print( pred,label)
print(acc_val/len(testloader))

0.47602570439940683


In [ ]:
dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=False, num_workers = 4)

In [ ]:
acc_val=0
for sample in dataloader:
    img,label = sample
    #plt.imshow(img.numpy().transpose((1,2,0)))
    img = img.to(device)
    label = label.to(device)
    label = Func.one_hot(label, num_classes=2).float()
    #label = label.unsqueeze(1)
    pred = net(img)
    acc_val += get_accuracy(pred.cpu(), label.cpu())
    #print(acc_val)
    #print( pred,label)
print(acc_val/len(dataloader))

0.9922654690618763
